In [1]:
import seqdata as sd
import seqpro as sp
import seqpro.xr as spxr
import numpy as np
import torch
from pathlib import Path
from pybedtools import BedTool
import os
import eugene as eu
import eugene.models
import eugene.train
import eugene.evaluate
from eugene.dataload._augment import RandomRC
from eugene import settings
from itertools import product
import glob

- For BPNet, grab ChIP track from ENCODE and put it in the `data_dir`.
- Window `trim_roi_jund_extended.bed` into 200 bp regions with `bedtools`. `trim_roi_jund_extended.bed` is already setup to be evenly divisible by 200.
- Add `kopp21` as a seqdataset.
- Include RandomRC for all models except kopp21_cnn

In [2]:
data_dir = Path('/cellar/users/aklie/data/eugene/kopp21/')
bam = data_dir/"dnase_stam_encode.bam"
targets = data_dir/"jund_raw_peaks.bed"
bed = data_dir/"trim_roi_jund_extended.bed"
final_bed = '/cellar/users/dlaub/projects/ML4GLand/use_cases/kopp21/data/jund_200bp.bed'
fasta = "/cellar/users/aklie/data/genomes/hg38/hg38.fa"
out = '/cellar/users/dlaub/projects/ML4GLand/use_cases/kopp21/kopp21.zarr'

In [3]:
# bed_200bp = BedTool(bed).makewindows(w=200, b=str(bed))  # type: ignore
# bed_200bp.saveas(final_bed)

In [4]:
# sdata = sd.read_bam(
#     'seq',
#     'cov',
#     out,
#     fasta,
#     [bam],
#     ['stam'],
#     final_bed,
#     batch_size=4096,
#     fixed_length=500,
#     threads_per_job=4,
#     alphabet='DNA',
#     overwrite=True
# )

In [5]:
sdata = sd.open_zarr(out)
sdata

<xarray.Dataset>
Dimensions:     (_sequence: 1013080, cov_sample: 1, _length: 500)
Coordinates:
  * cov_sample  (cov_sample) object 'stam'
Dimensions without coordinates: _sequence, _length
Data variables:
    chrom       (_sequence) object dask.array<chunksize=(63318,), meta=np.ndarray>
    chromEnd    (_sequence) int64 dask.array<chunksize=(63318,), meta=np.ndarray>
    chromStart  (_sequence) int64 dask.array<chunksize=(63318,), meta=np.ndarray>
    cov         (_sequence, cov_sample, _length) uint16 dask.array<chunksize=(4096, 1, 500), meta=np.ndarray>
    seq         (_sequence, _length) |S1 dask.array<chunksize=(4096, 500), meta=np.ndarray>
    strand      (_sequence) object dask.array<chunksize=(63318,), meta=np.ndarray>
Attributes:
    length_dim:    _length
    max_jitter:    0
    sequence_dim:  _sequence

In [6]:
targets_df = sd.read_bedlike(targets)
targets_df.head()

,chrom,chromStart,chromEnd
0,chr1,1000795,1001019
1,chr1,1024682,1024992
2,chr1,1032948,1033258
3,chr1,1064215,1064525
4,chr1,1291838,1292148


- Shrink ROIs by 150 on each side for getting labels

In [7]:
_sdata = sdata.copy()
_sdata['chromStart'] += 150
_sdata['chromEnd'] -= 150
sdata['target'] = sd.mark_sequences_for_classification(_sdata, targets_df, 'binary')

- Split data: test = chr3, val = chr2
- Load models of interest, in [EUGENe_paper repo](https://github.com/adamklie/EUGENe_paper/tree/revision/configs/kopp21)
    - Run training as a script
- Evaluate = predict(test)
- Look at EUGENe_paper repo for deets from eval and on

In [8]:
sdata['ohe_seq'] = spxr.ohe(sdata['seq'], sp.alphabets.DNA)

In [9]:
test = sdata.sel(_sequence=(sdata['chrom'] == 'chr3').compute())
train = sdata.assign(train_val=~(sdata['chrom'] == 'chr2').compute())

In [10]:
# Configure EUGENe
settings.dataset_dir = "/cellar/users/dlaub/projects/ML4GLand/use_cases/kopp21"
settings.output_dir = "/cellar/users/dlaub/projects/ML4GLand/EUGENe_paper/output/kopp21"
settings.logging_dir = "/cellar/users/dlaub/projects/ML4GLand/EUGENe_paper/logs/kopp21"
settings.config_dir = "/cellar/users/dlaub/projects/ML4GLand/EUGENe_paper/configs/kopp21"

In [11]:
def prep_new_model(
    config,
    seed,
):
    # Instantiate the model
    model = eu.models.load_config(
        config_path=config,
        seed=seed
    )
    
    # Initialize the model prior to conv filter initialization
    eu.models.init_weights(model)

    # Return the model
    return model 

In [ ]:
configs = ["fcn.yaml", "cnn.yaml", "hybrid.yaml", "kopp21_cnn.yaml"]
trials = 1

for config, trial in product(configs, range(1, trials+1)):
    model_name = config.split('.')[0]
    print(model_name)
    
    # Initialize the model
    model = prep_new_model(os.path.join(eu.settings.config_dir, config), seed=trial)
    
    transforms = {
        "target": lambda x: torch.tensor(x, dtype=torch.float32)
    }
    if model_name != 'kopp21_cnn.yaml':
        random_rc = RandomRC()
        def ohe_seq_transform(x):
            x = torch.tensor(x, dtype=torch.float32).swapaxes(1, 2)
            return random_rc(x)
        transforms["ohe_seq"] = ohe_seq_transform
    else:
        transforms["ohe_seq"] = lambda x: torch.tensor(x, dtype=torch.float32).swapaxes(1, 2)
        
    
    # Fit the model
    eu.train.fit_sequence_module(
        model,
        train,
        gpus=1,
        seq_key="ohe_seq",
        target_keys=["target"],
        in_memory=True,
        train_key="train_val",
        epochs=25,
        early_stopping_metric='val_loss_epoch',
        early_stopping_patience=5,
        batch_size=64,
        num_workers=4,
        prefetch_factor=2,
        drop_last=False,
        name=model_name,
        version=f"trial_{trial}",
        transforms=transforms,
        seed=trial,
    )
    
    del model

In [ ]:
print('hi')

In [ ]:
# Make an output directory for this dataset if it doesn't exist
if not os.path.exists(os.path.join(settings.output_dir, 'test')):
    os.makedirs(os.path.join(settings.output_dir, 'test'))

# Iterate over the models
for config in configs:
    model_name = config.split(".")[0]

    # Iterate over the trials
    for trial in range(1, trials+1):
    
        # Print the model name
        print(f"test {model_name} trial {trial}")

        # Grab the best model from that training run
        model_file = glob.glob(os.path.join(settings.logging_dir, model_name, f"trial_{trial}", "checkpoints", "*"))[0]
        model = eu.models.load_config(config_path=config)
        best_model = eu.models.SequenceModule.load_from_checkpoint(model_file, arch=model.arch)
        eu.evaluate.predictions_sequence_module(
            model=best_model,
            sdata=test,
            seq_key="ohe_seq",
            target_keys="target",
            gpus=1,
            batch_size=2048,
            num_workers=4,
            prefetch_factor=2,
            in_memory=True,
            transforms=transforms,
            file_label="test",
            name=model_name,
            version=f"trial_{trial}",
            prefix=f"{model_name}_trial_{trial}_"
        )

# Save the predictions
pred_keys = [k for k in test.data_vars.keys() if "predictions" in k]
(
    test[['chrom', 'chromStart', 'chromEnd', 'target', *pred_keys]]
    .to_dataframe()
    .to_csv(os.path.join(settings.output_dir, 'test', f"kopp21_test_predictions.tsv"), sep="\t", index=False)
)
sd.to_zarr(test, os.path.join(settings.output_dir, 'test', f"kopp21_test_predictions.zarr"), mode="w")